In [4]:
! pip install jaconv neologdn

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for neologdn: filename=neologdn-0.5.1-cp39-cp39-linux_x86_64.whl size=55312 sha256=77862a870896f2bbd145b5b2f27635d918f09f00a819d0d65a326401e249fbc5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/a4/b9/ca97bbfde3bf0646e407ada317bf9cbcf272b16529fca9d912
Successfully built neologdn


In [31]:
import pandas as pd 
import ast
import copy
import pickle
import jaconv
import unicodedata
import neologdn
import regex

In [32]:
#df_ = pd.read_excel('input_data/test_extra_df_all_.xlsx')
df_ = pd.read_excel('test_extra_df_all_.xlsx')

In [44]:
df=df_.copy()

In [45]:
def preprocess(text):
    # Neologd Normalization
    text = neologdn.normalize(text)
    # Normalization Form Compatibility Composition (NFKC)
    text = unicodedata.normalize("NFKC", text)
    # full-width characterization
    text = regex.sub(r'(\d)([,])(\d+)', r'\1\3', text)
    #text = text.replace(",", "、")
    #text = text.replace("，", "、")
    text = (jaconv.h2z(text, kana=True, digit=True, ascii=True))
    # remove full-width space
    text = text.replace("\u3000", "")
    return text

In [46]:
df['pick_word_check']=df['pick_word_check'].apply(lambda x: x.split(',') if type(x)==str else '該当なし' )
df['pick_word_check']=df['pick_word_check'].apply(lambda x:[preprocess(x_)  for x_ in x if type(x)!=str] )
for col in ['tokens','AE','pick_word','pick_words','label_words','pred_pick','result_pick','extra_pick']:
    #print(col)
    df[col] = df[col].apply(lambda x: ast.literal_eval(x))

## 方法2

In [47]:
def negate_ae(token,etracted_ae,negate_list):
    if token[-1]!='。':
        token=token+['。']
    token=['。']+token
    # コンマによって区切り
    split_index=[]
    split_comma=[]
    for i in range(len(token)):
        if token[i]=='。':
            split_index.append(i) 
            split_comma.append(True)
        if token[i]=='、'or token[i]=='（' or token[i]=='）' :
            split_index.append(i) 
            split_comma.append(False)

    #　コンマ間の文字数が5文字以下であれば"　消す
    delete_index=[]
    for i in range(len(split_index)-1):
        if split_index[i+1]-split_index[i]<6 and not split_comma[i+1]:
            delete_index.append(i+1)
    for i in reversed(delete_index):
        del split_index[i]    
    #print(split_index)
    # 分割された文で、否定語が文末[-2:]或は[-3:]にあるか判断　=>否定されたAE
    negate_ae=[]
    for s in range(len(split_index)-1):
        #print(token[split_index[s]+1:split_index[s+1]])
        for negate in negate_list:
            if negate in token[split_index[s+1]-3:split_index[s+1]] and not '重篤' in token[split_index[s]+1:split_index[s+1]]:
                #print('negate in this splited text:',negate)
                for i in range(split_index[s],split_index[s+1]):
                    if token[i] in etracted_ae:
                        negate_ae.append(token[i])
    negate_ae=list(set(negate_ae))
    return negate_ae       

In [23]:
"""
ない
はない
は認められない
は認めない  
は認めていない
には至っていないが
との見極めも出来ない
の異常はない
==>ない
"""
negate_list=[
    'なし',
    'なく',
    'ない',
    '否定的',
    'なかっ',
    'ず',
    '消失',
    '無し']

In [48]:
df['negate_ae']=df.apply(lambda x:negate_ae(x['tokens'],
                                            x['pick_word'],
                                            negate_list)    ,axis=1)

In [49]:
#　被っているケースを処理　e.g. pick_words:貧血  negate_ae:貧血進行
def filter_fun(pick_words,negate_ae):
    _=[] 
    for i in pick_words:
        for j in negate_ae:
            if j in i or i in j:
                _.append(i)    
    return _

df['negate_ae']=df.apply(lambda x:[i for i in x['pick_word'] if i in filter_fun(x['pick_word'],x['negate_ae'])]  ,axis=1)
df['negate_result']=df['negate_ae'].apply(lambda x: True if len(x)>0 else False  )

In [50]:
df[df['negate_result']==True][['row','pick_word','negate_ae']]

,row,pick_word,negate_ae
39,2021/11月にリムパーザとベバシズマブの維持療法を開始したものの、投与初期より嘔気(CT...,[嘔気],[嘔気]
52,24サイクル目まで肺臓炎悪化なく経過、イミフィンジ投与終了。,[肺臓炎],[肺臓炎]
57,プレドニゾロン5MGに減量。発熱、下痢、皮疹など増悪なく、骨髄抑制など副作用を認めず、胸部X...,"[皮疹, 下痢, 発熱]","[皮疹, 下痢, 発熱]"
305,MRIでは明らかな脳転移の所見なし。,[脳転移],[脳転移]
311,ORAPARIB400MG/日減量再開後、腎機能悪化、貧血G3(HB:7.9G/DL)で休薬...,"[貧血, 腎機能悪化]",[貧血]
...,...,...,...
7074,皮疹が改善したことから、タグリッソ錠80MG/日へ増量。この時点では胸部CTで部分奏効維持で...,"[心不全, 皮疹]",[心不全]
7320,貧血はなし。肝機能も異常無し。十全大補湯を処方。,[貧血],[貧血]
7387,処方後すぐに舌にピリピリとした痛みが出たが、悪心が強くて言わなかった。,[悪心],[悪心]
7464,150ミリを2錠1日2回飲んでいるが、薬があまりにも大きく飲み込まない。,[飲み],[飲み]


In [51]:
flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]
pick_word_flatten=flatten(df[df['negate_result']==True]['negate_ae'].tolist())
print('検出した行数：',len(df[df['negate_result']==True]))
print('検出したAE件数：',len(pick_word_flatten))

検出した行数： 161
検出したAE件数： 177


In [ ]:
df[df['negate_result']==True][['row','pick_word','negate_ae']].to_csv('否定処理方法2.csv', index=False)

## 新しいデータを作る

修正したデータを読み込み

In [62]:
df_ = pd.read_excel('否定処理方法2_.xlsx')

for col in ['pick_word','negate_ae_modifiied','negate_ae']:
    #print(col)
    df_[col] = df_[col].apply(lambda x: ast.literal_eval(x))
df_=df_[['row','pick_word','negate_ae_modifiied']]    

In [104]:
df_new=pd.merge(df,df_[['row','negate_ae_modifiied']],how='left',on='row')

In [105]:
df_new[df_new['negate_result']==True][['row','pick_word','negate_ae_modifiied']]

,row,pick_word,negate_ae_modifiied
39,2021/11月にリムパーザとベバシズマブの維持療法を開始したものの、投与初期より嘔気(CT...,[嘔気],[嘔気]
52,24サイクル目まで肺臓炎悪化なく経過、イミフィンジ投与終了。,[肺臓炎],[肺臓炎]
57,プレドニゾロン5MGに減量。発熱、下痢、皮疹など増悪なく、骨髄抑制など副作用を認めず、胸部X...,"[皮疹, 下痢, 発熱]","[皮疹, 下痢, 発熱]"
305,MRIでは明らかな脳転移の所見なし。,[脳転移],[脳転移]
311,ORAPARIB400MG/日減量再開後、腎機能悪化、貧血G3(HB:7.9G/DL)で休薬...,"[貧血, 腎機能悪化]",[削除なし]
...,...,...,...
7074,皮疹が改善したことから、タグリッソ錠80MG/日へ増量。この時点では胸部CTで部分奏効維持で...,"[心不全, 皮疹]",[心不全]
7320,貧血はなし。肝機能も異常無し。十全大補湯を処方。,[貧血],[貧血]
7387,処方後すぐに舌にピリピリとした痛みが出たが、悪心が強くて言わなかった。,[悪心],[悪心]
7464,150ミリを2錠1日2回飲んでいるが、薬があまりにも大きく飲み込まない。,[飲み],[飲み]


In [106]:
# pick_wordsで否定されたAEを削除
df_new['pick_words_new']=df_new[df_new['negate_result']==True].apply(lambda x:[i for i in x['pick_word'] if i not in x['negate_ae_modifiied']] ,axis=1)

In [108]:
# pick_wordsで否定されたAEを削除
df_new['pick_words_new']=df_new.apply(lambda x:[i for i in x['pick_word'] if i not in x['negate_ae_modifiied']] if x['negate_result']==True else x['pick_word'],axis=1)

In [109]:
df_new[df_new['negate_result']==True][['row','pick_word','pick_words_new','negate_ae_modifiied']]

,row,pick_word,pick_words_new,negate_ae_modifiied
39,2021/11月にリムパーザとベバシズマブの維持療法を開始したものの、投与初期より嘔気(CT...,[嘔気],[],[嘔気]
52,24サイクル目まで肺臓炎悪化なく経過、イミフィンジ投与終了。,[肺臓炎],[],[肺臓炎]
57,プレドニゾロン5MGに減量。発熱、下痢、皮疹など増悪なく、骨髄抑制など副作用を認めず、胸部X...,"[皮疹, 下痢, 発熱]",[],"[皮疹, 下痢, 発熱]"
305,MRIでは明らかな脳転移の所見なし。,[脳転移],[],[脳転移]
311,ORAPARIB400MG/日減量再開後、腎機能悪化、貧血G3(HB:7.9G/DL)で休薬...,"[貧血, 腎機能悪化]","[貧血, 腎機能悪化]",[削除なし]
...,...,...,...,...
7074,皮疹が改善したことから、タグリッソ錠80MG/日へ増量。この時点では胸部CTで部分奏効維持で...,"[心不全, 皮疹]",[皮疹],[心不全]
7320,貧血はなし。肝機能も異常無し。十全大補湯を処方。,[貧血],[],[貧血]
7387,処方後すぐに舌にピリピリとした痛みが出たが、悪心が強くて言わなかった。,[悪心],[],[悪心]
7464,150ミリを2錠1日2回飲んでいるが、薬があまりにも大きく飲み込まない。,[飲み],[],[飲み]


In [112]:
# pick_words_newでAEがない行を削除
cnt=0
index_list=[]
for idx,row in df_new.iterrows():
    if len(row['pick_words_new'])==0:
        cnt+=1
        index_list.append(row['level_0'])
print(cnt)
df_new= df_new[~df['level_0'].isin(index_list)]

97


In [116]:
#手作業での600件を置き換え
df_new['pick_words_new_2']=df_new.apply(lambda x:x['pick_word_check'] if x['extra_result']==True  else x['pick_words_new']  ,axis=1)

In [117]:
#手作業での600件中の記載なしを削除
cnt=0
index_list=[]
for idx,row in df_new.iterrows():
    if row['pick_words_new_2'][0]=='記載なし':
        cnt+=1
        index_list.append(row['level_0'])
print(cnt)
df_new= df_new[~df_new['level_0'].isin(index_list)]

22


In [118]:
df_new=df_new[['level_0','file_name','management_number','row', 'AE', 'LLT', 'PT','pick_words_new_2']].rename(columns={'pick_words_new_2':'pick_words_new'})
df_new

,level_0,file_name,management_number,row,AE,LLT,PT,pick_words_new
0,7,./input\タグリッソ_20211001-1231.xlsx,202167639,10回目、DAY245、爪囲炎改善したが再増悪。,[爪囲炎再増悪],['爪囲炎'],['爪囲炎'],[爪囲炎]
1,21,./input\イミフィンジ_20211001-1231.xlsx,202166641,18時頃突然の前胸部痛と冷汗あり。18:15頃に意識消失、CPR(心肺蘇生法)開始。酸素投与...,[心筋梗塞、心室細動],['心室細動'],['心室細動'],[心室細動]
2,28,./input\リムパーザ_20220201-0331.xlsx,210003833,1コース目D21、GRADE2好中球減少発現。,[GRADE2好中球減少],['好中球数減少'],['好中球数減少'],[GRADE2好中球減少]
3,33,./input\イミフィンジ_20211001-1231.xlsx,202171730,1コース目はシスプラチン投与の際のハイドレーションに伴い、一時的に体重増加(+3~4KG)、...,"[体重増加, 下肢浮腫, 腫瘍増大, 足がむくむ]","['体重増加', '下肢浮腫']","['体重増加', '末梢性浮腫']","[体重増加, 下肢浮腫]"
4,34,./input\イミフィンジ_20211001-1231.xlsx,210009515,1コース目カルボプラチン(AUC5)、エトポシド(100MG/M2)にてG4の好中球減少、F...,"[G4の好中球減少, FN]","['好中球数減少', '発熱性好中球減少症']","['好中球数減少', '発熱性好中球減少症']","[G4の好中球減少, 発熱性好中球減少症]"
...,...,...,...,...,...,...,...,...
7538,7515,./input\イミフィンジ_20211001-1231.xlsx,210002674,転倒、頭部を打撲、右眼窩外側壁骨折、顔面挫創、糖尿、アラニンアミノトランスフェラーゼ増加、高...,"[転倒, 頭部を打撲, 右眼窩外側壁骨折, 顔面挫創, 糖尿, アラニンアミノトランスフェラ...","['転倒', '頭部挫傷', '眼窩骨折', '挫創', '糖尿', 'アラニンアミノトラン...","['転倒', '挫傷', '顔面骨骨折', '創傷', '糖尿', 'アラニンアミノトランス...","[転倒, 頭部を打撲, 右眼窩外側壁骨折, 挫創, 糖尿, アラニンアミノトランスフェラーゼ..."
7539,7255,./input\リンパーザ_20211001-1231.xlsx,201936922,右側腹部痛と下腹部痛みあり。既往歴に憩室炎があるが、今は問題ない。,"[右側腹部痛, 下腹部痛]","['側腹部痛', '下腹部痛']","['側腹部痛', '下腹部痛']","[側腹部痛, 下腹部痛]"
7540,7512,./input\リンパーザ_20211001-1231.xlsx,202043354,貧血進行なし。リムパーザ錠休薬期間もあったが腫瘍マーカーは下降続けた。,[貧血],['貧血'],['貧血'],[腫瘍マーカー]
7541,7524,./input\タグリッソ_20220101-0430.xlsx,202168684,鑑別診断:潰瘍性大腸炎。,[薬剤性大腸炎],['大腸炎'],['大腸炎'],[大腸炎]


In [16]:
with open('input_data/df_row_new_2.pkl', 'wb') as file:
    pickle.dump(df_new, file)